In [1]:
# display, transform, read, split ...
import numpy as np
import matplotlib.pyplot as plt
import splitfolders

# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import  Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
# Dropout is a regularization technique to prevent overfitting by randomly setting a fraction of input units to 0 at each update during training.
from tensorflow.keras.layers import Dropout, BatchNormalization
# BatchNormalization normalizes the inputs of each layer to improve training speed and stability.
from tensorflow.keras.regularizers import l2

In [2]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# define classes name
class_names = ['Angry','Fear','Happy','Sad']
train_batches = ImageDataGenerator().flow_from_directory("./data-split/train/", target_size=(224,224), classes=['Angry','Fear','Happy','Sad'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory("./data-split/val/", target_size=(224,224),classes=['Angry','Fear','Happy','Sad'], batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory("./data-split/test/", target_size=(224,224), classes=['Angry','Fear','Happy','Sad'], batch_size=4)
vgg16_model = keras.applications.vgg16.VGG16()


Found 280 images belonging to 4 classes.
Found 80 images belonging to 4 classes.
Found 41 images belonging to 4 classes.


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the VGG16 model with weights pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4, activation='softmax')(x)  # 4 classes for 'Angry', 'Fear', 'Happy', 'Sad'

# Final model
model = Model(inputs=base_model.input, outputs=x)



In [5]:
train_generator = train_batches
valid_generator = valid_batches
test_generator = test_batches


In [7]:

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Calculate steps per epoch and validation steps
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# Initialize EarlyStopping callback
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    epochs=30,
    verbose=2,
    callbacks=[early_stopping]
)


Epoch 1/30
28/28 - 72s - 3s/step - accuracy: 0.4607 - loss: 2.4983 - val_accuracy: 0.3125 - val_loss: 3.9508
Epoch 2/30
28/28 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/30
28/28 - 78s - 3s/step - accuracy: 0.4607 - loss: 2.3181 - val_accuracy: 0.3375 - val_loss: 3.9410
Epoch 4/30
28/28 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/30
28/28 - 70s - 2s/step - accuracy: 0.4786 - loss: 2.1963 - val_accuracy: 0.3500 - val_loss: 3.8859
Epoch 6/30
28/28 - 0s - 612us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/30
28/28 - 68s - 2s/step - accuracy: 0.4893 - loss: 2.0878 - val_accuracy: 0.3500 - val_loss: 3.8252
Epoch 8/30
28/28 - 0s - 663us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/30
28/28 - 69s - 2s/step - accuracy: 0.5179 - loss: 1.9766 - val_accuracy: 0.3500 - val_loss: 3.8038
Epoch 10/30
28/28 - 0s - 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/30
28/28 - 73s - 3s/step - accuracy: 0.5071 - loss: 1.8801 - val_a

In [9]:
class_weights = {0: 1., 1: 1., 2: 1., 3: 1.5}  # Example: increase weight for the underrepresented class

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    epochs=30,
    verbose=2,
    class_weight=class_weights
)


NameError: name 'steps_per_epoch' is not defined

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_accuracy}, Test loss: {test_loss}")
